In [15]:
import os
import pickle

### Preprocess train-val-test splits of data

In [26]:
!python ./scripts/preprocess_data.py --inp_data_path ./data --dest_path ./data/experiment_tracking --fileout_vect ./artifacts/experiment_tracking/tfidf_vectorizer.pkl

INFO : loading pickle files
INFO : loading tf-idf vectorizer
INFO : preprocessing data splits
INFO : saving preprocessed pickle files


In [30]:
# check preprocessed files
print(os.listdir("./data/experiment_tracking/"))

# check tfidf vectorizer pickle file
print(os.listdir("./artifacts/experiment_tracking"))

['test.pkl', 'train.pkl', 'val.pkl']
['tfidf_vectorizer.pkl']


In [33]:
# check train data pickle
with open("./data/experiment_tracking/train.pkl", 'rb') as ftrain:
    dfx, dfy, dfx_cols, dfy_cols = pickle.load(ftrain)
print(type(dfx), type(dfy), type(dfx_cols), type(dfy_cols))
print(dfx.shape, dfy.shape, len(dfx_cols), len(dfy_cols))

<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'list'>
(32345, 796) (32345,) 796 1


### Experiment Tracking using mlflow

Run below command in terminal and open the url where mlflow is being served to track model training experiments:

mlflow ui --backend-store-uri sqlite:///mlflow.db

In [23]:
# baseline sklearn models
!python ./scripts/train.py

2024/08/18 22:18:30 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
d:\Installed Programs\miniconda3\envs\reviewsentinel\lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
d:\Installed Programs\miniconda3\envs\reviewsentinel\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setup

In [25]:
# random forest classifiers tuned using hyperopt
!python ./scripts/hpo.py


100%|██████████| 15/15 [2:31:16<00:00, 605.11s/trial, best loss: -0.7639681589301875] 


### Model Registry

In [119]:
# register top model from best 5 candidate models
!python ./scripts/register_model.py --tracked_exps hyperopt_rfc --candidate_exp candidate_mtest2

2024/08/19 21:48:27 INFO mlflow.tracking.fluent: Experiment with name 'candidate_mtest2' does not exist. Creating a new experiment.
2024/08/19 21:49:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "d:\Installed Programs\miniconda3\envs\reviewsentinel\lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml"
2024/08/19 21:50:06 WARNING mlflow.utils.validation: Tag value '[{"run_id": "5c5cc9ad10374fe48161b07b28ca7639", "artifact_path": "model", "utc_time_created": "2024-...' (5579 characters) is truncated to 5000 characters to meet the length limit.
2024/08/19 21:51:04 WARNING mlflow.utils.validation: Tag value '[{"run_id": "11d721bdda514331a3290b67fcc15bb5", "artifact_path": "model", "utc_time_created": 

In [ ]:
# code to delete registered model(and its versions); to be implemented later in the scripts

# Delete versions 1,2, and 3 of the model
# client = MlflowClient()
# versions = [1, 2, 3]
# for version in versions:
#     client.delete_model_version(
#         name="sk-learn-random-forest-reg-model", version=version
#     )

# Delete a registered model along with all its versions
# client.delete_registered_model(name="sk-learn-random-forest-reg-model")

In [ ]:
# code to remove deleted runs from mlflow directory; to be implemented later in the scripts
# ps: this only deletes the folders from windows filesystem but not its deleted entry from mlflow lifecycle

# import mlflow
# import shutil
    
# def get_run_dir(artifacts_uri):
#     return artifacts_uri[8:-10]
    
    
# def remove_run_dir(run_dir):
#     shutil.rmtree(run_dir, ignore_errors=True)
    
# experiment_id = 1
# deleted_runs = 2
    
# exp = mlflow.tracking.MlflowClient(tracking_uri='./mlflow/mlruns')
    
# runs = exp.search_runs(str(experiment_id), run_view_type=deleted_runs)
    
# _ = [remove_run_dir(get_run_dir(run.info.artifact_uri)) for run in runs]